In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import inspect

In [3]:
inspect.getfile(tf)

'/Users/rohan/.virtualenvs/jupyter-env/lib/python3.6/site-packages/tensorflow/__init__.py'

In [4]:
# model parameters.
num_epochs = 1000
total_series_lengths = 50000
truncated_backprop_length = 15
state_size = 4
num_classes = 2
echo_step = 3
batch_size = 5
num_batches = total_series_lengths // batch_size // truncated_backprop_length

In [5]:
def generate_data():
    x = np.array(np.random.choice(2, total_series_lengths))
    y = np.roll(x, echo_step)
    
    x = x.reshape((batch_size, -1))
    y = x.reshape((batch_size, -1))
    
    return x, y

In [6]:
x, y = generate_data()
print(x.shape, y.shape)

(5, 10000) (5, 10000)


In [8]:
batchx_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batchy_placeholder = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length])

init_state = tf.placeholder(tf.float32, [batch_size, state_size])

In [9]:
W = tf.Variable(np.random.rand(state_size+1, state_size), dtype=tf.float32)
b = tf.Variable(np.random.rand(1, state_size), dtype=tf.float32)

W2 = tf.Variable(np.random.rand(state_size, num_classes), dtype=tf.float32)
B2 = tf.Variable(np.random.rand(1, num_classes), dtype=tf.float32)

In [10]:
input_series = tf.unstack(batchx_placeholder, axis=1)
label_series = tf.unstack(batchy_placeholder, axis=1)

In [15]:
input_series[0].shape

TensorShape([Dimension(5)])